In [ ]:
import os
from collections import defaultdict

from PIL import Image
from Minibatch import MinibatchDiscrimination
import keras.backend as K
import numpy as np

In [ ]:
from keras.models import load_model
generator = load_model('models/generator_161.h5')

In [ ]:
def vis_square(data, padsize=1, padval=0):

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    return data

In [ ]:
latent_size = 110
generate_class = 2 # Choose an image class to generate, here 2 - birds
noise = np.random.normal(0, 0.5, (100, latent_size))
sampled_labels = np.array([
    [generate_class] * 10 for i in range(10)
]).reshape(-1, 1)
generated_images = generator.predict([noise, sampled_labels]).transpose(0, 2, 3, 1)
generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

In [ ]:
img = vis_square(generated_images)
Image.fromarray(img).save(
    'images/plot_class_{}_generated.png'.format(generate_class)) # store generated image 

In [ ]:
# For discriminator we used custom layer so we need to specify it here
disc = load_model('discriminator.h5',custom_objects={'MinibatchDiscrimination':MinibatchDiscrimination})